The goal of this project is to understand **weekly fluctuations** in sales and reveal the **hidden patterns in customer behavior**.

In [1]:
import numpy as np
import  pandas as pd

grocery1 = pd.read_csv('grocery_data1.csv', index_col = 0)
grocery2 = pd.read_csv('grocery_data2.csv', index_col = 0)

In [ ]:
# Preview the 2 datasets
print(grocery1.info())
print(grocery1.head())

print(grocery2.info())
grocery2.head()

In [2]:
# Parse each file separately before combining them (Because the DateRaw columns are in different format).
grocery1['DateRaw'] = pd.to_datetime(grocery1['DateRaw'], format = '%B %d, %Y')
grocery1['Time'] = pd.to_datetime(grocery1['Time'], format = '%H:%M:%S').dt.time
grocery1 = grocery1.rename(columns = {'DateRaw': 'Date'})

grocery2['DateRaw'] = pd.to_datetime(grocery2['DateRaw'])
grocery2['Time'] = pd.to_datetime(grocery2['Time'], format = '%H:%M:%S').dt.time
grocery2 = grocery2.rename(columns = {'DateRaw': 'Date'})

# Combine into one dataset
grocery = pd.concat([grocery1, grocery2], ignore_index = True)
print(grocery.info())
grocery.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   CustomerID     5200 non-null   int64         
 1   Date           5200 non-null   datetime64[ns]
 2   Time           5200 non-null   object        
 3   TransactionID  5200 non-null   int64         
 4   ProductName    5200 non-null   object        
 5   PriceUSD       5200 non-null   float64       
 6   Quantity       5200 non-null   int64         
 7   PaymentMethod  5200 non-null   object        
 8   Category       5200 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 365.8+ KB
None


,CustomerID,Date,Time,TransactionID,ProductName,PriceUSD,Quantity,PaymentMethod,Category
0,41,2023-06-28,20:00:00,2,Apples,5.64,5,Cash,Produce
1,170,2023-08-18,06:00:00,3,Apples,17.92,1,Mobile Payment,Produce
2,86,2023-08-18,09:00:00,4,Pasta,19.14,2,Mobile Payment,Grains
3,178,2023-08-06,02:00:00,5,Rice,0.76,4,Debit Card,Grains
4,87,2023-07-30,09:00:00,6,Chickpeas,11.30,1,Debit Card,Vegetarian


### Goal1
In order to understand consumer spending patterns, **what week** of the year during the time period given (June 1 to August 31) **had the smallest absolute deviation in sales value** compared to the mean weekly sales over that same time period?

In [3]:
# Find the typical week during the time period(June 1 to August 31)
# Add new columns Week & TotalSaleUSD
grocery['Week'] = grocery['Date'].dt.isocalendar().week
grocery['TotalSaleUSD'] = grocery['PriceUSD'] * grocery['Quantity']

# Find week sales
weekly_sales = grocery.groupby('Week')['TotalSaleUSD'].sum().reset_index()
weekly_sales = weekly_sales.rename(columns = {'TotalSaleUSD': 'WeeklyTotalSaleUSD'})

# Calculate the mean weekly sale
mean_sales = weekly_sales['WeeklyTotalSaleUSD'].mean()
print('Mean weekly sales:', mean_sales)

# Find the week with the smallest absolute deviation in sales value
weekly_sales['Diff'] = (weekly_sales['WeeklyTotalSaleUSD'] - mean_sales).abs()
smallest_sales_deviation = int(weekly_sales.sort_values('Diff').iloc[0]['Week'])
print('The week with the smallest absolute deviation in sales value:', smallest_sales_deviation)

Mean weekly sales: 11442.881428571429
The week with the smallest absolute deviation in sales value: 34


### Goal2
In order to evaluate individual buying preferences, **how many days went by between the three purchases** of cornflakes by CustomerID 107? 

In [6]:
# Find the Interpurchase time ---- how many days went by between each purchase of each product
# Add a new column DaysSinceLastPurchase showing the interpurchase time
grocery_sorted = grocery.sort_values(['CustomerID', 'ProductName', 'Date'])
grocery_sorted['DaysSinceLastPurchase'] = grocery_sorted.groupby(['CustomerID', 'ProductName'])['Date'].diff().dt.days.fillna(0).astype(int)
grocery_sorted.head()

,CustomerID,Date,Time,TransactionID,ProductName,PriceUSD,Quantity,PaymentMethod,Category,Week,TotalSaleUSD,DaysSinceLastPurchase
2941,1,2023-08-24,20:00:00,633,Apples,12.48,3,Mobile Payment,Produce,34,37.44,0
2849,1,2023-07-14,18:00:00,467,Beef,7.44,1,Cash,Meat,28,7.44,0
4633,1,2023-06-22,22:00:00,4115,Black Beans,11.21,1,Credit Card,Vegetarian,25,11.21,0
4287,1,2023-07-15,03:00:00,3384,Black Beans,3.56,5,Mobile Payment,Vegetarian,28,17.80,23
4552,1,2023-07-31,04:00:00,3965,Black Beans,7.98,4,Credit Card,Vegetarian,31,31.92,16


In [7]:
# Filter the cornflakes purchase record made by CustomerID 107 
cf_107 = grocery_sorted[(grocery_sorted['CustomerID'] == 107) & (grocery_sorted['ProductName'] == 'Cornflakes')]
print('The purchases of cornflakes made by customer 107 are 3 times, and the interpurchase time are: 6, 40')

The purchases of cornflakes made by customer 107 are 3 times, and the interpurchase time are: 6, 40


### Goal3
In order to tailor marketing strategies, **what hour** of the day (as a number on the 24 hour scale) **had the most hourly total sales?**

In [9]:
# Find the hour of the day with the highest total sales

# Find hourly total sales
hourly_sales = grocery.groupby('Time')['TotalSaleUSD'].sum().reset_index()
hourly_sales = hourly_sales.rename(columns = {'TotalSaleUSD': 'HourlyTotalSaleUSD'})

# Find the hour of the day with the most hourly total sales
most_hourly_sales = hourly_sales.sort_values('HourlyTotalSaleUSD', ascending = False).iloc[0]['Time']
print('The hour with the most hourly total sales:', most_hourly_sales)

The hour with the most hourly total sales: 22:00:00
